In [1]:
%load_ext autotime
import sys
sys.path.append('/Users/pradap/Documents/Research/Python-Package/anhaid/magellan/')

This quickstart guide explains how to match two tables. [[ Add more here ]]

In [2]:
# Import libraries
import magellan as mg
import os, sys

time: 1.47 s


In [3]:
print('python version: ' + sys.version + '\n')
print('magellan version: ' + mg.__version__ + '\n')

python version: 2.7.12 |Continuum Analytics, Inc.| (default, Jul  2 2016, 17:43:17) 
[GCC 4.2.1 (Based on Apple Inc. build 5658) (LLVM build 2336.11.00)]

magellan version: 0.1.0

time: 1.61 ms


In [4]:
# Workflow
## Read input tables
## Block tables to get candidate set
## Match tuple pairs in the candidate set

time: 818 µs


In [5]:
# Read tables
A = mg.read_csv_metadata('DBLP_ACM/DBLP_cleaned.csv', key='id')
B = mg.read_csv_metadata('DBLP_ACM/ACM_cleaned.csv', key='id')

time: 43.4 ms


In [6]:
A.head(2)

,id,title,authors,venue,year
0,journals/sigmod/Mackay99,Semantic Integration of Environmental Models for Application to Global Information Systems and D...,D. Scott Mackay,SIGMOD Record,1999
1,conf/vldb/PoosalaI96,Estimation of Query-Result Distribution and its Application in Parallel-Join Load Balancing,"Viswanath Poosala, Yannis E. Ioannidis",VLDB,1996


time: 36 ms


In [7]:
B.head(2)

,id,title,authors,venue,year
0,304586,The WASA2 object-oriented workflow management system,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
1,304587,A user-centered interface for querying distributed multimedia databases,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999


time: 13.7 ms


In [8]:
mg.get_key(A)

'id'

time: 4.96 ms


In [9]:
mg.get_key(B)

'id'

time: 3.66 ms


In [10]:
# Block tables to get the candidate set.
# Plan

# A, B ------ AttrEquivalence blocker [year] -----> C

time: 1.01 ms


In [11]:
# Create attribute equivalence blocker
ab = mg.AttrEquivalenceBlocker()
# Block tables using 'year' attribute: same year then include in the canidate set
C1 = ab.block_tables(A, B, 'year', 'year', 
                   l_output_attrs=['title', 'authors', 'year'],
                   r_output_attrs=['title', 'authors', 'year']
                   )

time: 269 ms


In [12]:
len(C1)

601284

time: 2.55 ms


In [13]:
C1.head(2)

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_year,rtable_title,rtable_authors,rtable_year
0,0,journals/sigmod/Mackay99,304586,Semantic Integration of Environmental Models for Application to Global Information Systems and D...,D. Scott Mackay,1999,The WASA2 object-oriented workflow management system,"Gottfried Vossen, Mathias Weske",1999
1,1,journals/sigmod/Mackay99,304587,Semantic Integration of Environmental Models for Application to Global Information Systems and D...,D. Scott Mackay,1999,A user-centered interface for querying distributed multimedia databases,"Isabel F. Cruz, Kimberly M. James",1999


time: 43.2 ms


In [14]:
# Debug blocker output
# Check whether the current blocker has dropped a lot of potential matches.
dbg = mg.debug_blocker(A, B, C1, output_size=200)

time: 3.52 s


In [15]:
dbg.head()

,_id,similarity,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_venue,rtable_title,rtable_authors,rtable_venue
0,0,0.875,journals/sigmod/Aberer02,776994,Book Review Column,Karl Aberer,SIGMOD Record,Book review column,Karl Aberer,ACM SIGMOD Record
1,1,0.875,journals/sigmod/Aberer02a,776994,Book Review Column,Karl Aberer,SIGMOD Record,Book review column,Karl Aberer,ACM SIGMOD Record
2,2,0.875,journals/sigmod/Aberer02b,604274,Book Review Column,Karl Aberer,SIGMOD Record,Book review column,Karl Aberer,ACM SIGMOD Record
3,3,0.875,journals/sigmod/Aberer03b,601865,Book review column,Karl Aberer,SIGMOD Record,Book review column,Karl Aberer,ACM SIGMOD Record
4,4,0.875,journals/sigmod/Aberer02,604274,Book Review Column,Karl Aberer,SIGMOD Record,Book review column,Karl Aberer,ACM SIGMOD Record


time: 32.6 ms


In [16]:
mg.view_table(dbg)

time: 4.06 s


In [17]:
# Update blocking sequence
# A, B --------------------->---- AE blocker ----->----C1 -------
#                                                                 |-----> C
# A, B --------------------->---- Overlap blocker ----->----C2---


time: 1.3 ms


In [18]:
ob = mg.OverlapBlocker()
C2 = ob.block_tables(A, B, 'authors', 'authors', 
                   l_output_attrs=['title', 'authors', 'year'],
                   r_output_attrs=['title', 'authors', 'year']
                   )

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:01


time: 2.56 s


In [19]:
len(C2)

307217

time: 2.67 ms


In [20]:
C2.head()

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_year,rtable_title,rtable_authors,rtable_year
0,0,journals/tods/LechtenborgerV03,304586,On the computation of relational view complements,"Vossen, Jens Gottfried Lechtenbrger",2003,The WASA2 object-oriented workflow management system,"Mathias Vossen, Gottfried Weske",1999
1,1,journals/sigmod/CherniakV03,304586,Reminiscences on Influential Papers,"Mitch Kenneth Cherniack, Gottfried A. Vossen Ross,",2003,The WASA2 object-oriented workflow management system,"Mathias Vossen, Gottfried Weske",1999
2,2,conf/sigmod/VossenW99,304586,The WASA2 Object-Oriented Workflow Management System,"Mathias Vossen Weske, Gottfried",1999,The WASA2 object-oriented workflow management system,"Mathias Vossen, Gottfried Weske",1999
3,3,journals/sigmod/McClatcheyV97,304586,Workshop on Workflow Management in Scientific and Engineering Applications - Report,"Vossen, McClatchey Gottfried Richard",1997,The WASA2 object-oriented workflow management system,"Mathias Vossen, Gottfried Weske",1999
4,4,journals/vldb/LaurentLSV01,304586,Monotonic complements for independent data warehouses,"Nicolas Dominique Lechtenbrger, Vossen, Jens Gottfried Laurent, Spyratos",2001,The WASA2 object-oriented workflow management system,"Mathias Vossen, Gottfried Weske",1999


time: 38.7 ms


In [21]:
# Combine blocker outputs
C = mg.combine_blocker_outputs_via_union([C1, C2])

time: 1.22 s


In [22]:
len(C)

875758

time: 2.7 ms


In [23]:
# debug again
dbg = mg.debug_blocker(A, B, C)

time: 6.04 s


In [24]:
mg.view_table(dbg)

time: 3.3 s


In [25]:
# Match tuple pairs in the candidate set
## Sample candidate set
## Label the sampled candidate set
## Split Labeled data into development and evaluation set
## Select the best learning based matcher using the development
## Evaluate the selected matcher using the evaluation setb

time: 1.5 ms


In [26]:
# Sample candidate set
S = mg.sample_table(C, 450)

time: 1.65 s


In [27]:
# Label S
L = mg.label_table(S, 'gold')

time: 2.9 s


In [32]:
# Load the pre-labeled data
L = mg.read_csv_metadata('DBLP_ACM/dblp_acm_demo_labels.csv', ltable=A, rtable=B)
len(L)

415

time: 10 ms


In [34]:
# Split G into development and evaluation
development_evaluation = mg.split_train_test(L, train_proportion=0.7)
development =  development_evaluation['train']
evaluation = development_evaluation['test']

time: 11.2 ms


In [36]:
# Selecting the best learning based matcher using the development set
## Create a set of ML-matchers
## Generate features 
## Extract feature vectors using the development set and the generated features
## Select the best learning-based matcher using k-fold cross validation
## Debug the selected matcher (and repeat the above steps)

time: 1.56 ms


In [37]:
# Create a set of ML-matchers
dt = mg.DTMatcher(name='DecisionTree')
svm = mg.SVMMatcher(name='SVM')
rf = mg.RFMatcher(name='RF')
nb = mg.NBMatcher(name='NB')
lg = mg.LogRegMatcher(name='LogReg')
ln = mg.LinRegMatcher(name='LinReg')

time: 3.59 ms


In [38]:
# Generate features
feature_table = mg.get_features_for_matching(A, B)

time: 64 ms


In [39]:
feature_table['feature_name']

0             title_title_jac_qgm_3_qgm_3
1         title_title_cos_dlm_dc0_dlm_dc0
2                         title_title_mel
3                    title_title_lev_dist
4                     title_title_lev_sim
5         authors_authors_jac_qgm_3_qgm_3
6     authors_authors_cos_dlm_dc0_dlm_dc0
7                     authors_authors_mel
8                authors_authors_lev_dist
9                 authors_authors_lev_sim
10                          year_year_exm
11                          year_year_anm
12                     year_year_lev_dist
13                      year_year_lev_sim
Name: feature_name, dtype: object

time: 7.27 ms


In [58]:
# Select year related features
feature_subset_iter1 = feature_table[10:14]

time: 1.5 ms


In [65]:
# Extract feature vectors
feature_vectors = mg.extract_feature_vecs(development, 
                            feature_table=feature_subset_iter1, 
                            attrs_after='gold')                                         

0%                          100%
[##############################] | ETA: 00:00:00

time: 353 ms



Total time elapsed: 00:00:00


In [71]:
# Impute feature vectors
feature_vectors = mg.impute_table(feature_vectors, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
                strategy='mean')

Inside else condition
time: 13 ms


In [72]:
feature_vectors.head(3)

,_id,ltable_id,rtable_id,year_year_exm,year_year_anm,year_year_lev_dist,year_year_lev_sim,gold
87,87,conf/sigmod/MinPC03,872775,1.0,1.000000,0.0,1.0,1
54,54,conf/sigmod/FernandezS96,775454,0.0,0.996505,4.0,0.0,0
355,355,journals/sigmod/Weikum02,601867,1.0,1.000000,0.0,1.0,1


time: 19 ms


In [73]:
# Select the best ML matcher using CV
result = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=feature_vectors, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        k=5,
        target_attr='gold', metric='f1') 

time: 199 ms


In [74]:
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<magellan.matcher.dtmatcher.DTMatcher object at 0x113d5b310>,5,0.827586,0.904762,0.886076,0.876712,0.837838,0.866595
1,RF,<magellan.matcher.rfmatcher.RFMatcher object at 0x113d5b210>,5,0.860759,0.818182,0.857143,0.891892,0.911392,0.867874
2,SVM,<magellan.matcher.svmmatcher.SVMMatcher object at 0x113d5b2d0>,5,0.840580,0.904110,0.852941,0.805195,0.938272,0.868219
3,NB,<magellan.matcher.nbmatcher.NBMatcher object at 0x113d5b190>,5,0.957746,0.891892,0.746269,0.902439,0.837838,0.867237
4,LogReg,<magellan.matcher.logregmatcher.LogRegMatcher object at 0x113d5b0d0>,5,0.888889,0.826667,0.861111,0.942529,0.816901,0.867219
5,LinReg,<magellan.matcher.linregmatcher.LinRegMatcher object at 0x113d5b050>,5,0.864198,0.793651,0.885714,0.883117,0.909091,0.867154


time: 24.1 ms


In [76]:
# Debug matcher
# # Split feature vectors into train and test
train_test = mg.split_train_test(feature_vectors, train_proportion=0.5)
train = train_test['train']
test = train_test['test']

time: 11.2 ms


In [77]:
# Debug decision tree using GUI
mg.vis_debug_dt(dt, train, test, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        target_attr='gold')

time: 21 s


In [78]:
# Select all features
feature_subset_iter2 = feature_table

time: 1.07 ms


In [88]:
# Get new set of features
feature_vectors = mg.extract_feature_vecs(development, feature_table=feature_subset_iter2, attrs_after='gold')
# Impute feature vectors
feature_vectors = mg.impute_table(feature_vectors, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
                strategy='mean')

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:01


Inside else condition
time: 1.24 s


In [82]:
# Apply cross validation to find if there is a better matcher
result = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=feature_vectors, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        target_attr='gold', metric='f1') 

time: 228 ms


In [83]:
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<magellan.matcher.dtmatcher.DTMatcher object at 0x113d5b310>,5,1.000000,0.950820,0.985507,1.000000,0.984615,0.984188
1,RF,<magellan.matcher.rfmatcher.RFMatcher object at 0x113d5b210>,5,0.982456,1.000000,0.984615,1.000000,0.968750,0.987164
2,SVM,<magellan.matcher.svmmatcher.SVMMatcher object at 0x113d5b2d0>,5,0.830189,0.953846,0.869565,0.945946,0.939394,0.907788
3,NB,<magellan.matcher.nbmatcher.NBMatcher object at 0x113d5b190>,5,1.000000,1.000000,0.984615,1.000000,0.983607,0.993644
4,LogReg,<magellan.matcher.logregmatcher.LogRegMatcher object at 0x113d5b0d0>,5,1.000000,0.952381,1.000000,0.985075,0.986667,0.984824
5,LinReg,<magellan.matcher.linregmatcher.LinRegMatcher object at 0x113d5b050>,5,0.969697,1.000000,0.982456,0.972222,0.965517,0.977979


time: 18.5 ms


In [84]:
# Evaluate matcher using the evaluation set
# 1. Extract feature vectors (using feature_subset_iter2)
# 2. Train matcher using feature vectors generated from the development set
# 3. Predict the evaluation set using the trained matcher
# 4. Evaluate the matches

time: 1.42 ms


In [89]:
# Get new set of features
feature_vectors_eval = mg.extract_feature_vecs(evaluation, 
                                               feature_table=feature_subset_iter2, 
                                               attrs_after='gold')
# Impute feature vectors
feature_vectors_eval = mg.impute_table(feature_vectors, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
                strategy='mean')

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


Inside else condition
time: 530 ms


In [90]:
# Train using feature vectors from the development set
rf.fit(table=feature_vectors, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
       target_attr='gold')
# Predict M 
predictions = rf.predict(table=feature_vectors_eval, 
                         exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
                         append=True, 
                         target_attr='predicted', 
                         inplace=False)

time: 40.7 ms


In [91]:
# Evaluate the result
eval_result = mg.eval_matches(predictions, 'gold', 'predicted')
mg.print_eval_summary(eval_result)

Precision : 100.0% (160/160)
Recall : 100.0% (160/160)
F1 : 100.0%
False positives : 0 (out of 160 positive predictions)
False negatives : 0 (out of 130 negative predictions)
time: 18.3 ms
